In [153]:
import csv
import requests
import json
import re
import urllib.request
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import clear_output
import matplotlib.pyplot as plt
from threading import Thread
from nltk.corpus import stopwords
import ast
import warnings
warnings.filterwarnings('ignore')


In [159]:
type = "decision"
keywords = "drug trafficking"
keywords = keywords.replace(' ', '%20')
base_url = "https://www.canlii.org"
search_url = "/en/search/ajaxSearch.do?type=decision&sort=decisionDateDesc&text="
page_start = 1
page_max = 9999999999999

https://www.canlii.org/en/search/ajaxSearch.do?type=decision&sort=decisionDateDesc&text=drug%20trafficking&page=1

https://www.canlii.org/en/search/ajaxSearch.do?type=decision&startDate=1800-01-01&endDate=2021-12-31&sort=decisionDateDesc&text=drug%20trafficking&page=1

In [ ]:
start_date = '1800-01-01'
end_date = '2021-12-31'

first_url_chunk = '/en/search/ajaxSearch.do?type=decision&'
date_url_chunk = 'startDate=' + start_date + '&endDate=' + end_date
last_url_chunk = '&sort=decisionDateDesc&text=' + keywords + '&page=' + str(page)


In [ ]:
'https://www.canlii.org/en/search/ajaxSearch.do?type=decision&startDate=1800-01-01&endDate=2021-12-31&sort=decisionDateDesc&text=drug%20trafficking&page=1'




In [257]:
def get_config_for_page(page):
    try:
        query_url = base_url + search_url + keywords + '&page=' + str(page)
        response = requests.get(query_url)
        responseJSON = json.loads(response.text)
        response = responseJSON
        return response
    except urllib.error.HTTPError as err:
        print(err)   
        
def get_config_for_page_with_date(page, date):
    start_date = '1800-01-01'
    end_date = date
    first_url_chunk = '/en/search/ajaxSearch.do?type=decision&'
    date_url_chunk = 'startDate=' + start_date + '&endDate=' + end_date
    last_url_chunk = '&sort=decisionDateDesc&text=' + keywords + '&page=' + str(page)
    try:
        query_url = base_url + first_url_chunk + date_url_chunk + last_url_chunk
        response = requests.get(query_url)
        responseJSON = json.loads(response.text)
        response = responseJSON
        return response
    except urllib.error.HTTPError as err:
        print(err)  

In [258]:
def create_case_file_from_config(case):    
    path = case['path']
    date = case['judgmentDate']
    title = case['title']
    concatId = case['concatId']
    topics = case['topics']
    row = {'id': concatId, 'date': date, 'path': path, 'title': title, 'topics': topics}
    
    return row

In [262]:
%%time
complete_config = []

previous_config = ''
query_more = True
more_than_fifty = False
page = 1
total_pages = 1
date_to_search_before = ''

case_df = pd.DataFrame(columns = ['id', 'date', 'path', 'title', 'topics'])

while query_more:
        
    if not more_than_fifty:
        print(f"Searching page {page} with no date specified")
        config = get_config_for_page(page)
    else:
        print(f"Seaching page {page} before date {date_to_search_before}")
        config = get_config_for_page_with_date(page, date_to_search_before)
    
    
    
    
    if config != previous_config:
        for case in config['results']:
            row = create_case_file_from_config(case)
            case_df = case_df.append(row, ignore_index=True)
        previous_config = config
        fifty_counter +=1
    else:
        print('is same')
        query_more = False
        
    print(f"Case df has {len(case_df)} cases, total pages: {total_pages}")
    
    page += 1
    total_pages += 1
    if page == 20:
        date_to_search_before = case_df[-1:]['date'].iloc[0]
        more_than_fifty = True
        page = 1    
        
print(f"Handled {total_pages} pages.")

case_df


Searching page 1 with no date specified
Case df has 25 cases, total pages: 1
Searching page 2 with no date specified
Case df has 50 cases, total pages: 2
Searching page 3 with no date specified
Case df has 75 cases, total pages: 3
Searching page 4 with no date specified
Case df has 100 cases, total pages: 4
Searching page 5 with no date specified
Case df has 125 cases, total pages: 5
Searching page 6 with no date specified
Case df has 150 cases, total pages: 6
Searching page 7 with no date specified
Case df has 175 cases, total pages: 7
Searching page 8 with no date specified
Case df has 200 cases, total pages: 8
Searching page 9 with no date specified
Case df has 225 cases, total pages: 9
Searching page 10 with no date specified
Case df has 250 cases, total pages: 10
Searching page 11 with no date specified
Case df has 275 cases, total pages: 11
Searching page 12 with no date specified
Case df has 300 cases, total pages: 12
Searching page 13 with no date specified
Case df has 325 case

Case df has 2600 cases, total pages: 104
Seaching page 10 before date 2019-11-19
Case df has 2625 cases, total pages: 105
Seaching page 11 before date 2019-11-19
Case df has 2650 cases, total pages: 106
Seaching page 12 before date 2019-11-19
Case df has 2675 cases, total pages: 107
Seaching page 13 before date 2019-11-19
Case df has 2700 cases, total pages: 108
Seaching page 14 before date 2019-11-19
Case df has 2725 cases, total pages: 109
Seaching page 15 before date 2019-11-19
Case df has 2750 cases, total pages: 110
Seaching page 16 before date 2019-11-19
Case df has 2775 cases, total pages: 111
Seaching page 17 before date 2019-11-19
Case df has 2800 cases, total pages: 112
Seaching page 18 before date 2019-11-19
Case df has 2825 cases, total pages: 113
Seaching page 19 before date 2019-11-19
Case df has 2850 cases, total pages: 114
Seaching page 1 before date 2019-06-14
Case df has 2875 cases, total pages: 115
Seaching page 2 before date 2019-06-14
Case df has 2900 cases, total 

Case df has 5150 cases, total pages: 206
Seaching page 17 before date 2017-12-01
Case df has 5175 cases, total pages: 207
Seaching page 18 before date 2017-12-01
Case df has 5200 cases, total pages: 208
Seaching page 19 before date 2017-12-01
Case df has 5225 cases, total pages: 209
Seaching page 1 before date 2017-06-29
Case df has 5250 cases, total pages: 210
Seaching page 2 before date 2017-06-29
Case df has 5275 cases, total pages: 211
Seaching page 3 before date 2017-06-29
Case df has 5300 cases, total pages: 212
Seaching page 4 before date 2017-06-29
Case df has 5325 cases, total pages: 213
Seaching page 5 before date 2017-06-29
Case df has 5350 cases, total pages: 214
Seaching page 6 before date 2017-06-29
Case df has 5375 cases, total pages: 215
Seaching page 7 before date 2017-06-29
Case df has 5400 cases, total pages: 216
Seaching page 8 before date 2017-06-29
Case df has 5425 cases, total pages: 217
Seaching page 9 before date 2017-06-29
Case df has 5450 cases, total pages: 

Case df has 7700 cases, total pages: 308
Seaching page 5 before date 2015-06-24
Case df has 7725 cases, total pages: 309
Seaching page 6 before date 2015-06-24
Case df has 7750 cases, total pages: 310
Seaching page 7 before date 2015-06-24
Case df has 7775 cases, total pages: 311
Seaching page 8 before date 2015-06-24
Case df has 7800 cases, total pages: 312
Seaching page 9 before date 2015-06-24
Case df has 7825 cases, total pages: 313
Seaching page 10 before date 2015-06-24
Case df has 7850 cases, total pages: 314
Seaching page 11 before date 2015-06-24
Case df has 7875 cases, total pages: 315
Seaching page 12 before date 2015-06-24
Case df has 7900 cases, total pages: 316
Seaching page 13 before date 2015-06-24
Case df has 7925 cases, total pages: 317
Seaching page 14 before date 2015-06-24
Case df has 7950 cases, total pages: 318
Seaching page 15 before date 2015-06-24
Case df has 7975 cases, total pages: 319
Seaching page 16 before date 2015-06-24
Case df has 8000 cases, total pag

Case df has 10250 cases, total pages: 410
Seaching page 12 before date 2013-03-26
Case df has 10275 cases, total pages: 411
Seaching page 13 before date 2013-03-26
Case df has 10300 cases, total pages: 412
Seaching page 14 before date 2013-03-26
Case df has 10325 cases, total pages: 413
Seaching page 15 before date 2013-03-26
Case df has 10350 cases, total pages: 414
Seaching page 16 before date 2013-03-26
Case df has 10375 cases, total pages: 415
Seaching page 17 before date 2013-03-26
Case df has 10400 cases, total pages: 416
Seaching page 18 before date 2013-03-26
Case df has 10425 cases, total pages: 417
Seaching page 19 before date 2013-03-26
Case df has 10450 cases, total pages: 418
Seaching page 1 before date 2012-10-25
Case df has 10475 cases, total pages: 419
Seaching page 2 before date 2012-10-25
Case df has 10500 cases, total pages: 420
Seaching page 3 before date 2012-10-25
Case df has 10525 cases, total pages: 421
Seaching page 4 before date 2012-10-25
Case df has 10550 ca

Case df has 12775 cases, total pages: 511
Seaching page 18 before date 2011-02-02
Case df has 12800 cases, total pages: 512
Seaching page 19 before date 2011-02-02
Case df has 12825 cases, total pages: 513
Seaching page 1 before date 2010-09-16
Case df has 12850 cases, total pages: 514
Seaching page 2 before date 2010-09-16
Case df has 12875 cases, total pages: 515
Seaching page 3 before date 2010-09-16
Case df has 12900 cases, total pages: 516
Seaching page 4 before date 2010-09-16
Case df has 12925 cases, total pages: 517
Seaching page 5 before date 2010-09-16
Case df has 12950 cases, total pages: 518
Seaching page 6 before date 2010-09-16
Case df has 12975 cases, total pages: 519
Seaching page 7 before date 2010-09-16
Case df has 13000 cases, total pages: 520
Seaching page 8 before date 2010-09-16
Case df has 13025 cases, total pages: 521
Seaching page 9 before date 2010-09-16
Case df has 13050 cases, total pages: 522
Seaching page 10 before date 2010-09-16
Case df has 13075 cases, 

Case df has 15300 cases, total pages: 612
Seaching page 5 before date 2008-04-03
Case df has 15325 cases, total pages: 613
Seaching page 6 before date 2008-04-03
Case df has 15350 cases, total pages: 614
Seaching page 7 before date 2008-04-03
Case df has 15375 cases, total pages: 615
Seaching page 8 before date 2008-04-03
Case df has 15400 cases, total pages: 616
Seaching page 9 before date 2008-04-03
Case df has 15425 cases, total pages: 617
Seaching page 10 before date 2008-04-03
Case df has 15450 cases, total pages: 618
Seaching page 11 before date 2008-04-03
Case df has 15475 cases, total pages: 619
Seaching page 12 before date 2008-04-03
Case df has 15500 cases, total pages: 620
Seaching page 13 before date 2008-04-03
Case df has 15525 cases, total pages: 621
Seaching page 14 before date 2008-04-03
Case df has 15550 cases, total pages: 622
Seaching page 15 before date 2008-04-03
Case df has 15575 cases, total pages: 623
Seaching page 16 before date 2008-04-03
Case df has 15600 cas

Case df has 17825 cases, total pages: 713
Seaching page 11 before date 2005-02-18
Case df has 17850 cases, total pages: 714
Seaching page 12 before date 2005-02-18
Case df has 17875 cases, total pages: 715
Seaching page 13 before date 2005-02-18
Case df has 17900 cases, total pages: 716
Seaching page 14 before date 2005-02-18
Case df has 17925 cases, total pages: 717
Seaching page 15 before date 2005-02-18
Case df has 17950 cases, total pages: 718
Seaching page 16 before date 2005-02-18
Case df has 17975 cases, total pages: 719
Seaching page 17 before date 2005-02-18
Case df has 18000 cases, total pages: 720
Seaching page 18 before date 2005-02-18
Case df has 18025 cases, total pages: 721
Seaching page 19 before date 2005-02-18
Case df has 18050 cases, total pages: 722
Seaching page 1 before date 2004-02-18
Case df has 18075 cases, total pages: 723
Seaching page 2 before date 2004-02-18
Case df has 18100 cases, total pages: 724
Seaching page 3 before date 2004-02-18
Case df has 18125 c

Case df has 20350 cases, total pages: 814
Seaching page 17 before date 1997-12-24
Case df has 20375 cases, total pages: 815
Seaching page 18 before date 1997-12-24
Case df has 20400 cases, total pages: 816
Seaching page 19 before date 1997-12-24
Case df has 20425 cases, total pages: 817
Seaching page 1 before date 1995-01-06
Case df has 20450 cases, total pages: 818
Seaching page 2 before date 1995-01-06
Case df has 20475 cases, total pages: 819
Seaching page 3 before date 1995-01-06
Case df has 20500 cases, total pages: 820
Seaching page 4 before date 1995-01-06
Case df has 20525 cases, total pages: 821
Seaching page 5 before date 1995-01-06
Case df has 20550 cases, total pages: 822
Seaching page 6 before date 1995-01-06
Case df has 20575 cases, total pages: 823
Seaching page 7 before date 1995-01-06
Case df has 20600 cases, total pages: 824
Seaching page 8 before date 1995-01-06
Case df has 20625 cases, total pages: 825
Seaching page 9 before date 1995-01-06
Case df has 20650 cases, 

,id,date,path,title,topics
0,2022bcsc993,2022-06-13,/en/bc/bcsc/doc/2022/2022bcsc993/2022bcsc993.html,Robinson v Gallamore,[]
1,2022abqb412,2022-06-13,/en/ab/abqb/doc/2022/2022abqb412/2022abqb412.html,R v Ayyazi,[]
2,2022abca203,2022-06-10,/en/ab/abca/doc/2022/2022abca203/2022abca203.html,R v Yanga,[]
3,2022onca459,2022-06-10,/en/on/onca/doc/2022/2022onca459/2022onca459.html,R. v. McMahon,"[Appeal, Conviction, Criminal, Sentencing]"
4,2022bcca215,2022-06-10,/en/bc/bcca/doc/2022/2022bcca215/2022bcca215.html,R. v. Graham,[]
...,...,...,...,...,...
22280,1923bcca1000011,1923-01-09,/en/bc/bcca/doc/1923/1923canlii420/1923canlii4...,Morley v. Oak Bay,[]
22281,1922skca1000152,1922-01-16,/en/sk/skca/doc/1922/1922canlii647/1922canlii6...,"Rex v. Regina Wine and Spirit, Ltd.; Rex v. Pr...",[Interpretation]
22282,1922skca1000088,1922-01-16,/en/sk/skca/doc/1922/1922canlii210/1922canlii2...,Rex v. Prairie Drug Co.;Rex v. Regina Wine & S...,[Interpretation]
22283,1913nbca1000000,1913-04-18,/en/nb/nbca/doc/1913/1913canlii423/1913canlii4...,Rex v. Holyoke; Ex Parte McIntyre,[]


In [263]:
case_df.to_csv('canadian court cases/cases.csv', index=False)

In [265]:
case_df.to_pickle("canadian court cases/cases.pkl")

In [264]:
case_df

,id,date,path,title,topics
0,2022bcsc993,2022-06-13,/en/bc/bcsc/doc/2022/2022bcsc993/2022bcsc993.html,Robinson v Gallamore,[]
1,2022abqb412,2022-06-13,/en/ab/abqb/doc/2022/2022abqb412/2022abqb412.html,R v Ayyazi,[]
2,2022abca203,2022-06-10,/en/ab/abca/doc/2022/2022abca203/2022abca203.html,R v Yanga,[]
3,2022onca459,2022-06-10,/en/on/onca/doc/2022/2022onca459/2022onca459.html,R. v. McMahon,"[Appeal, Conviction, Criminal, Sentencing]"
4,2022bcca215,2022-06-10,/en/bc/bcca/doc/2022/2022bcca215/2022bcca215.html,R. v. Graham,[]
...,...,...,...,...,...
22280,1923bcca1000011,1923-01-09,/en/bc/bcca/doc/1923/1923canlii420/1923canlii4...,Morley v. Oak Bay,[]
22281,1922skca1000152,1922-01-16,/en/sk/skca/doc/1922/1922canlii647/1922canlii6...,"Rex v. Regina Wine and Spirit, Ltd.; Rex v. Pr...",[Interpretation]
22282,1922skca1000088,1922-01-16,/en/sk/skca/doc/1922/1922canlii210/1922canlii2...,Rex v. Prairie Drug Co.;Rex v. Regina Wine & S...,[Interpretation]
22283,1913nbca1000000,1913-04-18,/en/nb/nbca/doc/1913/1913canlii423/1913canlii4...,Rex v. Holyoke; Ex Parte McIntyre,[]


In [136]:
def get_case_text(path):
    text = ''
    try:
        query_url = base_url + path
        response = requests.get(query_url)
        case_text = response.text
        soup = BeautifulSoup(case_text, 'html.parser')
        documentContainer = soup.find('div', {'id': 'documentContainer'})
        documentContent = documentContainer.find('div', {'id': 'documentContent'})
        originalDocument = documentContent.find('div', {'id': 'originalDocument'})
        originalDocument_divs = originalDocument.find_all('div', attrs={'class': re.compile('^WordSection.*')})
        for div in originalDocument_divs:
            text+= div.text
        return text
    except Exception as err:
        print(err)

case_text = get_case_text('/en/ab/abpc/doc/2021/2021abpc264/2021abpc264.html')

In [157]:
case_df = pd.DataFrame(columns = ['id', 'date', 'path', 'title'])
  
for case in response['results']:
    row = create_case_file(case)
    case_df = case_df.append(row, ignore_index=True)
    
case_df

,id,date,path,title
0,2021abpc264,2021-10-15,/en/ab/abpc/doc/2021/2021abpc264/2021abpc264.html,R v Bay
1,2021onsc7615,2021-10-14,/en/on/onsc/doc/2021/2021onsc7615/2021onsc7615...,R. v. Mohamed
2,2021bcsc2199,2021-10-14,/en/bc/bcsc/doc/2021/2021bcsc2199/2021bcsc2199...,R. v Lambert
3,2021oncj559,2021-10-14,/en/on/oncj/doc/2021/2021oncj559/2021oncj559.html,R. v. Stewart
4,2021mbpc49,2021-10-14,/en/mb/mbpc/doc/2021/2021mbpc49/2021mbpc49.html,R. v. Krywonizka
5,2021bcpc250,2021-10-14,/en/bc/bcpc/doc/2021/2021bcpc250/2021bcpc250.html,T.L.C. v. K.J.H.
6,2021ntsc34,2021-10-13,/en/nt/ntsc/doc/2021/2021nwtsc34/2021nwtsc34.html,R v Omar
7,2021onsc6798,2021-10-13,/en/on/onsc/doc/2021/2021onsc6798/2021onsc6798...,R. v. Moseley
8,2021abqb811,2021-10-13,/en/ab/abqb/doc/2021/2021abqb811/2021abqb811.html,R v Wallis
9,2021onltb1013815,2021-10-12,/en/on/onltb/doc/2021/2021canlii139717/2021can...,2295477 Ontario Inc. v Chen


{'concatId': '2021abpc264', 'path': '/en/ab/abpc/doc/2021/2021abpc264/2021abpc264.html', 'title': 'R v Bay', 'reference': '2021 ABPC 264 (CanLII)', 'judgmentDate': '2021-10-15', 'collectionTitle': 'Provincial Court', 'jurisdictionTitle': 'Alberta', 'jurisdictionId': 'ab', 'keywords': 'firearms — locker — garage — gun safe — evidence', 'topics': [], 'firstSyncDate': 'Oct 18, 2021, 7:34:35 PM', 'updateDate': 'Oct 18, 2021, 12:00:00 AM', 'templateName': 'CanliiCaselawIndexedResult.ftl', 'resultNumber': 476, 'snippet': '[…]\xa0\xa0 Additionally, she is charged with two counts simple possession of a controlled substance pursuant to the Controlled <b>Drugs</b> and Substances Act.  \xa0\xa0[…]\xa0\xa0 [48] Counts 1-12 are charges pursuant to section 99(1) of the Criminal Code commonly known as Weapons <b>Trafficking</b>.  \xa0\xa0[…]\xa0\xa0 Possession of those <b>drugs</b> is an offence contrary the Controlled <b>Drugs</b> and Substances Act. These <b>drugs</b> were found in Ms. Bay’s purse 

In [135]:
for case in response['results']:
    path = case['path']
    print(path)
    text = get_case_text(path)
    print(len(text))

/en/ab/abpc/doc/2021/2021abpc264/2021abpc264.html
36397
/en/on/onsc/doc/2021/2021onsc7615/2021onsc7615.html
44049
/en/bc/bcsc/doc/2021/2021bcsc2199/2021bcsc2199.html
26381
/en/on/oncj/doc/2021/2021oncj559/2021oncj559.html
69750
/en/mb/mbpc/doc/2021/2021mbpc49/2021mbpc49.html
41945
/en/bc/bcpc/doc/2021/2021bcpc250/2021bcpc250.html
232942
/en/nt/ntsc/doc/2021/2021nwtsc34/2021nwtsc34.html
50607
/en/on/onsc/doc/2021/2021onsc6798/2021onsc6798.html
13783
/en/ab/abqb/doc/2021/2021abqb811/2021abqb811.html
60001
/en/on/onltb/doc/2021/2021canlii139717/2021canlii139717.html
11309
/en/on/onltb/doc/2021/2021canlii140664/2021canlii140664.html
8830
/en/ca/fct/doc/2021/2021fc1047/2021fc1047.html
39674
/en/ca/fct/doc/2021/2021fc1062/2021fc1062.html
56727
/en/on/onsc/doc/2021/2021onsc7025/2021onsc7025.html
39140
/en/sk/skpc/doc/2021/2021skpc48/2021skpc48.html
40332
/en/bc/bcca/doc/2021/2021bcca401/2021bcca401.html
18314
/en/bc/bcsc/doc/2021/2021bcsc2090/2021bcsc2090.html
70660
/en/ab/abqb/doc/2021/2021a

In [91]:
first['path']

'/en/ab/abqb/doc/2022/2022abqb412/2022abqb412.html'

In [123]:
text = ''
soup = BeautifulSoup(example_case_text, 'html.parser')
documentContainer = soup.find('div', {'id': 'documentContainer'})
documentContent = documentContainer.find('div', {'id': 'documentContent'})
originalDocument = documentContent.find('div', {'id': 'originalDocument'})
originalDocument_divs = originalDocument.find_all('div', attrs={'class': re.compile('^WordSection.*')})

for div in originalDocument_divs:
    text+= div.text
    print(len(text))
# len(originalDocument_divs)
# print(originalDocument_divs[0].text)

226
30387


In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
            soup_content = soup.find("div", {"id": 'content'})
            if df.iloc[i]['verdict_type'] == 'uitspraak':
                statement_content = soup_content.find('div', {'class': 'uitspraak'})
            elif df.iloc[i]['verdict_type'] == 'conclusie':
                statement_content = soup_content.find('div', {'class': 'conclusie'})
            
            sections = {}
            for el in statement_content.find_all(attrs={'class': 'section'}):
                section_title = clean_string(el.find('h2').text)
                print(section_title)
                sections[section_title] = el